## 11.2 准备文本数据

还是再重复一遍,现有的 nlp 模型只是在文本中提取模型,而不是真正意义上的理解.

如同前面 imdb 数据集的例子,模型无法处理输入的文本,需要将文本转换为数值张量.实现文本到张量的方法很多,但是一般都有相同的步骤.

- 首先是文本标准化,将其转化为小写字母,删除标点符号等.
- 将文本分割为单元(这个过程称之为标记),例如字符/词/词组等.
- 将每个这样的标记转成数字张量,通常进行这一步之前,需要将数据中所有标记进行索引.

![from_text_to_vectors](from_text_to_vectors.png)


## 文本标准化

考虑下面两个句子

- "sunset came. i was staring at the Mexico sky. Isnt nature splendid??"
- "Sunset came; I stared at the Mexico sky. Isn’t nature splendid?"

上面两个句子人读取之后会认为这是几乎一致的两个句子,但是转换成字符串比较看看?

- i 和 I
- Mexico 和 Mexico 都是墨西哥但是拼写不同
- Isnt 和 Isn’t
- stared 和 staring

机器学习无法知道 i 和 I 完全相同,就是大小写区别,é 是带重音的 e.这些规则和常识,机器无法使用,所以需要对文本进行标准化,尽力抹除这样的差异.



文本标准化也是特征工程的一种,旨在消除不希望模型处理的编码差异.这个过程并非机器学习独有,其他的工程例如搜索引擎也会进行.

最简单,最广泛的标准化方法: 转换成小写并删除标点符号.在样例上应用.

- "sunset came i was staring at the mexico sky isnt nature splendid"
- "sunset came i stared at the méxico sky isnt nature splendid"

比原来接近一些了,还有一种是特殊字符替换.将特殊字符替换为标准字符.这里的就是 e 替代 é,ae 替换 æ 等.

- "sunset came i was staring at the mexico sky isnt nature splendid"
- "sunset came i stared at the mexico sky isnt nature splendid"

抹除了一个单词的差异.

还有一种方法是非常高级的标准化,在机器学习背景的 nlp 中更少使用,将术语标准化.将一个术语(一个动词的多种形式/不同连接方式)转换成单一的表示.

- caught 或者 been catching 可以被替换为 catch.
- cats 可以替换为 cat.
- was staring 和 stared 可以被替换为 stare.

进行这样的替换,很可能两个意思相近的句子最终会是完全相同的编码.

- "sunset came i [stare] at the mexico sky isnt nature splendid"


文字标准化后,模型才可能在较少数据量情况下有更好的泛化能力.模型不需要理解 mexico 和 méxico 都是墨西哥,也不需要理会 Sunset 和 sunset 都是日落.当然文字标准化可能会抹除一些信息,(如果是在小说/文章中可能是一些非常精妙的文字描写--私注),所以进行文本标准化要始终看上下文的需要,加入是从采访文章中提取信息,那 `?` 肯定不能直接扔了吧? 必须单独做一个重点标记.


## 文本拆分(文本标记化)

一旦文本标准化结束,此时需要开始将文本分割为单元(标记),这个步骤称为标记化.一般有 3 种方式

- 单词级别标记化: 将每一个单词转换成一个向量,单词是由空格或标点隔离的子串.还有一个变种是进一步分割子串,例如将 staring 再拆分成 star + ing,将 called 拆分成 call + ed.
- N-gram 标记化: 将连续 N 个词组转换成一个向量.the cat 或者 he was 就是 2-gram 标记.
- 字符级别标记化: 每一个字符都 是自己的标记,转换成一个向量.这样的方案很少使用,一般只会出现在文生成/语音识别的场景下.

一般而言我们只会用到单词级别标记化和 N-gram 标记化.有两种文本处理模型:

- 序列模型: 关注单词的顺序,一般使用单词级别的标记化.
- 词袋模型: 不关心单词顺序,只将所有标记当作一个集合对待.一般使用 N-gram 标记化(N-gram 实际上注入了少量人为的单词顺序信息)

我们这一节都会看到.


### 理解 n-gram 和词袋

n-gram 是由 N 个(或少于 N)连续字词组成的组,这些组从连续的句子中拆分.注意一点, n-gram 的概念也可以应用于字符.

考虑下面的例子: `the cat sat on the mat`

拆分成 2-gram

```text
{"the", "the cat", "cat", "cat sat", "sat",
 "sat on", "on", "on the", "the mat", "mat"}
```

3-grams

```text
{"the", "the cat", "cat", "cat sat", "the cat sat",
 "sat", "sat on", "on", "cat sat on", "on the",
 "sat on the", "the mat", "mat", "on the mat"}
```

将原句整个拆分后的集合称为 2-gram 词袋 和 3-gram 词袋.不是列表和序列,没有顺序,这样拆分标记化的方法称为 bag-of-N-grams.

`bag-of-N-grams` 会破坏文本数据的顺序,往往用在浅层的语言处理模型,而不是深度学习中.深度学习并没有使用 n-gram 这样人工的特征工程,取而代之的是使用分层的特征学习,cnn rnn 和 Transformer 都能通过观察连续的单词/序列在没有明确输入这些组的情况下学习单词和字符组的表示.


## 单词索引

一旦单词标记完成,我们需要将每一个标记转换成数字表示.有一种偷懒的办法,将每个标记进行散列运算,取散列值.但是在实践中,通常是为所有标记建立索引,并为每个条目分配一个唯一索引(整数).


大致过程类似下面:

```py
vocabulary = {}
for text in dataset:
    text = standardize(text)
    tokens = tokenize(text)
    for token in tokens:
        if token not in vocabulary:
            vocabulary[token] = len(vocabulary)
```

下一步是将这些整数转换成神经网络可以处理的矢量,例如使用 one-hot 编码.

```py
def one_hot_encode_token(token):
    vector = np.zeros((len(vocabulary),))
    token_index = vocabulary[token]
    vector[token_index] = 1
    return vector
```


注意在这一步,通常会将标记数量限制在 2万~3万之间.任何文本数据集都有大量独特的词汇,其中大部分只出现一两次,对这些罕见词汇进行索引会造成过大的特征空间,这样特征空间的大部分特征几乎没有什么信息量.

在第4 5 章的 imdb 数据集上,我们使用数据时有参数 `num_words=10000`,我们只取了训练数据词频前 1000 的单词.

有一个重要的细节,我们不应该忽视,在词汇表中查找标记,标记可能不存在.训练数据中包含 `cherimoya` 的可能性不大.执行 `token_index = vocabulary["cherimoya"]` 会导致 `KeyError`.我们需要一个编外的记号,在查找失败时返回.实践中这个标记通常被称为 `OOV` 标记,值通常为 `UNK`,通常索引位置是 1.

至于 `OOV` 的索引位置是 1,那么 0 被什么占用了?

0 的位置是 `mask` 标记.

- `OOV` 意味着查不到对应标记.
- `mask` 则表示这不是有意义单词,只是用来填充数组的值.神经网络要求输入的数据规模相同,对较短的序列需要填充数据.`[5,  7, 124, 4, 89]` 和 `[8, 34,  21]` 下,`[8, 34,  21]` 会被填充到 `[8, 34,  21, 0,  0]`.


## 使用 TextVectorization 层

上一小节的内容可以概括成下面的代码


In [2]:
import string


class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)

    def tokenize(self, text):
        text = self.standardize(text)
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}  #最开始 0 1位置初始化
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i, "[UNK]")
            for i in int_sequence)  #未查找成功


vectorizer = Vectorizer()
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
vectorizer.make_vocabulary(dataset)

In [3]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [4]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


上述代码并不复杂,但是在实践中性能可能并不好(毕竟是纯 py 的实现),keras 当然有实现相同功能的方式 -> TextVectorization 层.


In [5]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorization = TextVectorization(output_mode="int", )


TextVectorization 会完成上面一小节的过程

- 默认文本标准化,转换成小写字母删除标点符号
- 默认文本分割标记化,按照空格分割并标记化.
- 可以传入自定义的标准和和标记化函数


In [6]:
import re
import string
import tensorflow as tf


def custom_standardization_fn(string_tensor):
    lowercase_string = tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(lowercase_string,
                                    f"[{re.escape(string.punctuation)}]", "")


def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)


text_vectorization = TextVectorization(
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn,
)

**注意** TextVectorization 层的自定义函数,操作的对象必须是 `tf.strings` 而不是普通的 string 类型.


In [7]:
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]
text_vectorization.adapt(dataset)

对文本语料库词汇进行索引,TextVectorization 层的 adapt 方法.只需要传入一个字符串的 dataset 的对象或者字符串列表.


In [8]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

`get_vocabulary` 可以将检索整个词汇表.如果需要将单词编码成整数序列,非常方便.

- 返回的前两个位置是 mask 标记和 OOV 标记
- 词汇表是按照单词的频率排序的,因此取高频词非常方便.


In [9]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)  #编码
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [10]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)]
                            for i in encoded_sentence)  #解码
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


### TextVectorization 层的使用

TextVectorization 层进行的是像字典一样的查询操作,只能在 cpu 上执行,不能在 gpu tpu 上执行.如果你的模型在 gpu 上训练,且包含了 TextVectorization 层,那 TextVectorization 层会在 cpu 上运行,结果再返回 gpu 这样会引起性能瓶颈.

有两种调用 TextVectorization 层的方式

- 使用 tf.data 管道,连接 TextVectorization 层.

  ```py
  int_sequence_dataset = string_dataset.map(text_vectorization)
  ```

- 直接将 TextVectorization 像普通 keras 层一样直接加入模型

  ```py
  text_input = keras.Input(shape=(), dtype="string")
  vectorized_text = text_vectorization(text_input)
  embedded_input = keras.layers.Embedding(...)(vectorized_text)
  output = ...
  model = keras.Model(text_input, output)
  ```

这两种方式有着决定性的差别,模型训练时各层之间是同步关系,如果 TextVectorization 是模型的一部分,那么这意味在 gpu 每一次的训练迭代中,模型其他部分将不得不等待 cpu 处理完这个批次 TextVectorization 的输出.之后 gpu 训练,cpu 再次空闲.但是如果使用 tf.data 管道,这意味着 cpu 能异步处理 TextVectorization 的输出,当 gpu 得到数据开始训练时,cpu 能直接处理下一批次的数据,缓存到 tf.data 管道.

因此我们在本章的例程中使用的是 tf.data 管道.但是如果你的模型是在 cpu 上训练的,完全不会有同步调度的问题,两种方式始终都会是 100% cpu 利用.
